In [158]:
import numpy as np
import pandas as pd
import plotly.express as px
from itertools import product

pd.options.plotting.backend = "plotly"

In [159]:
# https://docs.screeps.com/api/#Creep
CREEP_TICKS_LIFE = 1500
MAX_BODY_SIZE = 50
MAX_DISTANCE = 50
DEFAULT_DISTANCE = 30

In [160]:
# Assuming one moving part for each carry part
actual_max_size = round(MAX_BODY_SIZE / 3)
work_parts = [*range(1, actual_max_size)]
carry_parts = [*range(1, actual_max_size)]
distances = [*range(0, MAX_DISTANCE)]
carry_parts

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [161]:
def harvested_tick(work):
    return work * 2

def capacity(carry):
    return carry * 50

def ticks_to_full(work, carry):
    return capacity(carry) / harvested_tick(work)

def round_trip(work, carry, distance):
    return (distance * 2) + ticks_to_full(work, carry) + 1

def body_cost(work, carry):
    move = carry
    return work * 100 + carry * 50 + move * 50

def profit(work, carry, distance):
    return round(CREEP_TICKS_LIFE / round_trip(work, carry, distance) * capacity(carry) - body_cost(work, carry))

def cost_benefit(work, carry, distance):
    return profit(work, carry, distance) / body_cost(work, carry)

In [162]:
cost_benefit(5, 5, 5)

9.417

In [163]:
cost_benefit_heatmap = list(map(lambda wc: [str(wc), wc[0], wc[1], wc[2], cost_benefit(*wc)], product(work_parts, carry_parts, distances)))

In [165]:
df = pd.DataFrame(cost_benefit_heatmap, columns=["body","work_parts", "carry_parts", "distance", "cost_benefit"])
df["ratio"] = df['carry_parts'] / df['work_parts']
df = df.sort_values(by="cost_benefit")
df

,body,work_parts,carry_parts,distance,cost_benefit,ratio
12049,"(16, 1, 49)",16,1,49,-0.561176,0.062500
12048,"(16, 1, 48)",16,1,48,-0.552353,0.062500
12047,"(16, 1, 47)",16,1,47,-0.542941,0.062500
11249,"(15, 1, 49)",15,1,49,-0.534375,0.066667
12046,"(16, 1, 46)",16,1,46,-0.533529,0.062500
...,...,...,...,...,...,...
10500,"(14, 3, 0)",14,3,0,19.819412,0.214286
12100,"(16, 3, 0)",16,3,0,19.821053,0.187500
7250,"(10, 2, 0)",10,2,0,19.833333,0.200000
11300,"(15, 3, 0)",15,3,0,19.833333,0.200000


In [168]:
px.line(df, x="body", y="cost_benefit", color="distance")

In [170]:
px.scatter(df, x="ratio", y="cost_benefit", color="distance")